In [1]:
# import os
# os.environ['PIP_DISABLE_PIP_VERSION_CHECK'] = '1'

# !pip install -q --upgrade pip
# !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
# !pip install -q --upgrade numpy scipy scikit-learn
# !pip install -q timm pycocotools faster-coco-eval
# !pip install -q --upgrade transformers lightly-train
# !pip install -q wandb
# !pip install -q -U "numpy<2.1" matplotlib --force-reinstall --no-cache-dir

In [2]:
# %%writefile trainer_convnext.py
# import os
# import math
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch import Tensor
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from torch.utils.data import DataLoader
# from transformers import AutoModel, AutoConfig
# from PIL import Image
# from pycocotools.coco import COCO
# from torchvision import transforms as T
# from tqdm import tqdm
# import wandb
# import datetime
# import torch.distributed as dist
# from torch.nn.parallel import DistributedDataParallel as DDP
# from torch.utils.data.distributed import DistributedSampler

# class HuggingFaceTeacherWrapper(nn.Module):
#     def __init__(self, model_id: str, token: str = None):
#         super().__init__()
#         if int(os.environ.get("RANK", 0)) == 0:
#             print(f"Loading teacher model '{model_id}' from Hugging Face...")
#         config = AutoConfig.from_pretrained(model_id, token=token)
#         self._model = AutoModel.from_pretrained(model_id, token=token)
#         self.is_vit = "vit" in config.model_type.lower()
#         self._feature_dim = (
#             self._model.config.hidden_size
#             if self.is_vit
#             else self._model.config.hidden_sizes[-1]
#         )
#         if int(os.environ.get("RANK", 0)) == 0:
#             print(f"Detected {'ViT' if self.is_vit else 'ConvNeXT'} architecture. Feature dim: {self._feature_dim}")

#     def feature_dim(self) -> int:
#         return self._feature_dim

#     def forward(self, x: Tensor) -> Tensor:
#         outputs = self._model(pixel_values=x, output_hidden_states=True)
#         if self.is_vit:
#             patch_tokens = outputs.last_hidden_state[:, 1:, :]
#             b, s, d = patch_tokens.shape
#             h = w = int(math.sqrt(s))
#             return patch_tokens.permute(0, 2, 1).reshape(b, d, h, w)
#         return outputs.hidden_states[-1]

# class CocoDetectionForDistill(torch.utils.data.Dataset):
#     def __init__(self, root, ann_file, transforms):
#         self.root = root
#         self.coco = COCO(ann_file)
#         self.ids = list(sorted(self.coco.imgs.keys()))
#         self.transforms = transforms

#     def __getitem__(self, index):
#         img_id = self.ids[index]
#         path = self.coco.loadImgs(img_id)[0]["file_name"]
#         img = Image.open(os.path.join(self.root, path)).convert("RGB")
#         return self.transforms(img), 0

#     def __len__(self):
#         return len(self.ids)

# def setup_ddp():
#     dist.init_process_group(backend="nccl")
#     torch.cuda.set_device(int(os.environ["LOCAL_RANK"]))

# def cleanup_ddp():
#     dist.destroy_process_group()


# def main_training_function(rank, world_size, config):
#     device = rank
    
#     is_main_process = (rank == 0)
    
#     if is_main_process:
#         print(f"Running DDP on {world_size} GPUs.")
#         timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
#         run_name = f"run_ddp_{timestamp}_lr{config['learning_rate']}_bs{config['batch_size_per_gpu']}"
#         try:
#             from kaggle_secrets import UserSecretsClient
#             from huggingface_hub import login
#             secrets = UserSecretsClient()
#             hf_token = secrets.get_secret("HUGGINGFACE_TOKEN")
#             wandb_key = secrets.get_secret("WANDB_API_KEY")
#             login(token=hf_token)
#             wandb.login(key=wandb_key)
#             wandb.init(project=config["wandb_project"], config=config, name=run_name)
#         except Exception:
#             hf_token = None
#             print("Could not log in, continuing without W&B.")
#     else:
#         hf_token = None

#     dist.barrier()
    
#     teacher_model = HuggingFaceTeacherWrapper(config["teacher_hf_id"], token=hf_token).to(device)
#     teacher_model.eval()

#     if is_main_process:
#         print("Đang tải student model trên tiến trình chính...")
#         torch.hub.load("lyuwenyu/RT-DETR", "rtdetrv2_l", pretrained=True, trust_repo=True)

#     dist.barrier()

#     student_hub_model = torch.hub.load("lyuwenyu/RT-DETR", "rtdetrv2_l", pretrained=True, trust_repo=True)
#     student_model = student_hub_model.model.to(device)

#     with torch.no_grad():
#         x = torch.randn(1, 3, 640, 640).to(device)
#         student_channels = student_model.encoder(student_model.backbone(x))[-1].shape[1]
#     teacher_channels = teacher_model.feature_dim()
#     projection_layer = nn.Conv2d(student_channels, teacher_channels, kernel_size=1).to(device)

#     student_model = DDP(student_model, device_ids=[device])
#     projection_layer = DDP(projection_layer, device_ids=[device])
    
#     transforms = T.Compose([
#         T.Resize((640, 640)), T.ToTensor(),
#         T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ])

#     train_dataset = CocoDetectionForDistill(
#         root=os.path.join(config["dataset_dir"], "train2017"),
#         ann_file=os.path.join(config["dataset_dir"], "annotations/instances_train2017.json"),
#         transforms=transforms
#     )
#     val_dataset = CocoDetectionForDistill(
#         root=os.path.join(config["dataset_dir"], "val2017"),
#         ann_file=os.path.join(config["dataset_dir"], "annotations/instances_val2017.json"),
#         transforms=transforms
#     )
#     if is_main_process:
#         print(f"Data loaded: {len(train_dataset)} training images, {len(val_dataset)} validation images.")

#     train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank, shuffle=True)
#     val_sampler = DistributedSampler(val_dataset, num_replicas=world_size, rank=rank, shuffle=False)

#     train_loader = DataLoader(
#         train_dataset, batch_size=config["batch_size_per_gpu"],
#         shuffle=False, num_workers=config["num_workers"], pin_memory=True, drop_last=True, sampler=train_sampler
#     )
#     val_loader = DataLoader(
#         val_dataset, batch_size=config["batch_size_per_gpu"],
#         shuffle=False, num_workers=config["num_workers"], pin_memory=True, drop_last=False, sampler=val_sampler
#     )

#     params = list(student_model.module.backbone.parameters()) + \
#              list(student_model.module.encoder.parameters()) + \
#              list(projection_layer.module.parameters())
             
#     optimizer = torch.optim.AdamW(params, lr=config["learning_rate"], weight_decay=config["weight_decay"])
#     criterion = nn.MSELoss()
#     scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=config['scheduler_factor'], patience=config['scheduler_patience'], verbose=is_main_process)

#     if is_main_process and wandb.run:
#         wandb.watch((student_model, projection_layer), log="all", log_freq=100)
    
#     best_val_loss = float('inf')
#     early_stopping_counter = 0

#     if is_main_process:
#         print("Starting training...")
        
#     for epoch in range(config["epochs"]):
#         train_sampler.set_epoch(epoch)
        
#         start = time.time()
#         student_model.train()
#         projection_layer.train()
#         total_train_loss = 0.0
        
#         train_iterator = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']} [Train]") if is_main_process else train_loader

#         for images, _ in train_iterator:
#             images = images.to(device)
#             with torch.no_grad():
#                 teacher_features = teacher_model(images)
#             student_features = student_model.module.encoder(student_model.module.backbone(images))[-1]
#             projected = projection_layer(student_features)
#             teacher_resized = F.interpolate(teacher_features, size=projected.shape[-2:], mode="bilinear", align_corners=False)
#             loss = criterion(projected, teacher_resized)
            
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             total_train_loss += loss.item()
        
#         train_loss_tensor = torch.tensor(total_train_loss).to(device)
#         dist.all_reduce(train_loss_tensor, op=dist.ReduceOp.SUM)
#         avg_train_loss = train_loss_tensor.item() / (len(train_loader) * world_size)

#         student_model.eval()
#         projection_layer.eval()
#         total_val_loss = 0.0
        
#         val_iterator = tqdm(val_loader, desc=f"Epoch {epoch+1}/{config['epochs']} [Val]") if is_main_process else val_loader
#         with torch.no_grad():
#             for images, _ in val_iterator:
#                 images = images.to(device)
#                 teacher_features = teacher_model(images)
#                 student_features = student_model.module.encoder(student_model.module.backbone(images))[-1]
#                 projected = projection_layer(student_features)
#                 teacher_resized = F.interpolate(teacher_features, size=projected.shape[-2:], mode="bilinear", align_corners=False)
#                 loss = criterion(projected, teacher_resized)
#                 total_val_loss += loss.item()
                
#         val_loss_tensor = torch.tensor(total_val_loss).to(device)
#         dist.all_reduce(val_loss_tensor, op=dist.ReduceOp.SUM)
#         avg_val_loss = val_loss_tensor.item() / (len(val_loader) * world_size)
        
#         if is_main_process:
#             duration = time.time() - start
#             print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Duration: {duration:.2f}s")
#             if wandb.run:
#                 wandb.log({"epoch": epoch + 1, "train/avg_loss": avg_train_loss, "val/avg_loss": avg_val_loss, "time/epoch_s": duration, "train/epoch_lr": optimizer.param_groups[0]['lr']})

#             scheduler.step(avg_val_loss)

#             if avg_val_loss < best_val_loss:
#                 best_val_loss = avg_val_loss
#                 early_stopping_counter = 0
#                 print(f"Validation loss improved to {best_val_loss:.4f}. Saving best model...")
#                 best_weights = {**student_model.module.backbone.state_dict(), **student_model.module.encoder.state_dict()}
#                 torch.save({'model': best_weights}, config["best_weights_filename"])

#             else:
#                 early_stopping_counter += 1
#                 print(f"Validation loss did not improve. Early stopping counter: {early_stopping_counter}/{config['early_stopping_patience']}")

#         stop_training = torch.tensor(1 if early_stopping_counter >= config['early_stopping_patience'] else 0, device=device)
#         dist.all_reduce(stop_training, op=dist.ReduceOp.MAX)
#         if stop_training.item() == 1:
#             if is_main_process:
#                 print("Early stopping triggered. Training finished.")
#             break
            
#     if is_main_process:
#         print("\nDistillation finished.")
#         final_weights = {**student_model.module.backbone.state_dict(), **student_model.module.encoder.state_dict()}
#         torch.save({'model': final_weights}, config["final_weights_filename"])
#         print(f"Saved final epoch weights to '{config['final_weights_filename']}'")
#         print(f"Best weights were saved to '{config['best_weights_filename']}' with val_loss: {best_val_loss:.4f}")
#         if wandb.run:
#             wandb.summary["best_val_loss"] = best_val_loss
#             wandb.finish()


# if __name__ == "__main__":
#     setup_ddp()
#     rank = int(os.environ["RANK"])
#     world_size = int(os.environ["WORLD_SIZE"])

#     DATASET_DIR = "/kaggle/input/dsp-pre-final/processed_taco_coco"
#     config = {
#         "learning_rate": 1e-4, "epochs": 50, "batch_size_per_gpu": 16,
#         "num_workers": 2, "weight_decay": 1e-5,
#         "teacher_hf_id": "facebook/dinov3-convnext-base-pretrain-lvd1689m",
#         "dataset_dir": DATASET_DIR,
#         "scheduler_patience": 3, "scheduler_factor": 0.1,
#         "early_stopping_patience": 7,
#         "best_weights_filename": "distilled_rtdetr_convnext_teacher_BEST.pth",
#         "final_weights_filename": "distilled_rtdetr_convnext_teacher_FINAL.pth",
#         "wandb_project": "Distill-RTDETR-ConvNeXt-Teacher",
#     }
    
#     main_training_function(rank, world_size, config)
    
#     cleanup_ddp()

In [3]:
# !torchrun --nproc_per_node=2 trainer_convnext.py

In [4]:
# %%writefile trainer_vit.py
# import os
# import sys
# import shutil
# import torch
# import wandb
# import lightly_train
# from lightly_train.model_wrappers import RTDETRModelWrapper
# import datetime

# def main_training_function(config):
#     is_main_process = os.environ.get("LOCAL_RANK", "0") == "0"

#     if is_main_process:
#         timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
#         run_name = f"run_ddp_{timestamp}_lr{config['learning_rate']}_bs{config['batch_size_per_gpu']}"
#         try:
#             from kaggle_secrets import UserSecretsClient
#             wandb_key = UserSecretsClient().get_secret("WANDB_API_KEY")
#             wandb.login(key=wandb_key)
#         except Exception as e:
#             print(f"W&B secrets not available. Skipping login. Error: {e}")

#         if os.path.exists(config['output_dir']):
#             print(f"Output directory '{config['output_dir']}' already exists. Deleting it.")
#             shutil.rmtree(config['output_dir'])

#     if not is_main_process:
#         run_name = ""

#     if torch.distributed.is_initialized():
#         torch.distributed.barrier()

#     if is_main_process:
#         print("Initializing Student Model (RT-DETR)...")
#     student_hub_model = torch.hub.load('lyuwenyu/RT-DETR', 'rtdetrv2_l', pretrained=True, trust_repo=True)
#     wrapped_student = RTDETRModelWrapper(student_hub_model.model)
    
#     callbacks_config = {
#         "model_checkpoint": {
#             "dirpath": os.path.join(config['output_dir'], 'checkpoints'),
#             "filename": 'best-model-{epoch}-{validation_loss:.4f}',
#             "monitor": 'val_loss',
#             "mode": 'min',
#             "save_top_k": 1,
#         },
#         "learning_rate_monitor": {}
#     }
    
#     global_batch_size = config['batch_size_per_gpu'] * config['num_gpus']

#     if is_main_process:
#         print("Starting distillation with lightly_train.train()...")
#         print(f"Global batch size: {global_batch_size} ({config['batch_size_per_gpu']} per GPU)")

#     lightly_train.train(
#         model=wrapped_student,
#         method="distillationv1",
#         method_args={
#             "teacher": config['teacher_name'],
#             "teacher_url": config['teacher_url'],
#         },
#         data=[config['train_dir'], config['val_dir']],
#         out=config['output_dir'],
#         epochs=config['epochs'],
#         batch_size=global_batch_size,
#         num_workers=config['num_workers'],
#         optim=config['optimizer_name'],
#         optim_args={"lr": config['learning_rate'], "weight_decay": config['weight_decay']},
#         callbacks=callbacks_config,
#         loggers={
#             "wandb": {
#                 "project": config['wandb_project'],
#                 "name": run_name,
#             }
#         },
#         devices=config['num_gpus'],
#         strategy='ddp_find_unused_parameters_true',
#         accelerator='gpu'
#     )
#     if is_main_process:
#         print("\nDistillation finished.")
#         print(f"Best model checkpoint saved in directory: {os.path.join(config['output_dir'], 'checkpoints')}")
        
# if __name__ == '__main__':
#     DINOV3_VIT_TEACHER_URL = ""
#     try:
#         from kaggle_secrets import UserSecretsClient
#         DINOV3_VIT_TEACHER_URL = UserSecretsClient().get_secret("DINOV3_TEACHER_URL")
#     except Exception as e:
#          print(f"Could not read secret 'DINOV3_VIT_URL'. Please set it manually. Error: {e}")

#     if not DINOV3_VIT_TEACHER_URL:
#         print("ERROR: Save your token key into kaggle secret")
#     else:
#         BASE_DIR = "/kaggle/input/dsp-pre-final/processed_taco_coco"
#         TRAIN_DIR = os.path.join(BASE_DIR, "train2017")
#         VAL_DIR = os.path.join(BASE_DIR, "val2017")
        
#         config = {
#             "num_gpus": 2,
#             "epochs": 50, "batch_size_per_gpu": 8, "num_workers": 2,
#             "optimizer_name": "adamw", "learning_rate": 1e-4, "weight_decay": 1e-5,
#             "early_stopping_patience": 7,
#             "teacher_name": "dinov3/vitb16", 
#             "teacher_url": DINOV3_VIT_TEACHER_URL,
#             "train_dir": TRAIN_DIR,
#             "val_dir": VAL_DIR,
#             "output_dir": "out/distill_vit_lightly",
#             "wandb_project": "Distill-RTDETR-Distill-VIT"
#         }
        
#         main_training_function(config)

In [5]:
# !torchrun --nproc_per_node=2 trainer_vit.py

# Finetune RT-DETR

In [6]:
# !rm -rf /kaggle/working/RT-DETR

In [7]:
%cd /kaggle/working
!git clone https://github.com/lyuwenyu/RT-DETR.git
!cd RT-DETR/rtdetrv2_pytorch && pip install -r requirements.txt -q
!cp "/kaggle/working/distilled_rtdetr_convnext_teacher_BEST.pth" "./RT-DETR/"
!pip install -q protobuf==3.20.3
!pip install -q tensorboard
!pip install -q --upgrade pip
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install -q --upgrade numpy scipy scikit-learn
!pip install -q timm pycocotools faster-coco-eval
!pip install -q --upgrade transformers lightly-train
!pip install -q wandb
!pip install -q -U "numpy<2.1" matplotlib --force-reinstall --no-cache-dir

/kaggle/working
Cloning into 'RT-DETR'...
remote: Enumerating objects: 1100, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 1100 (delta 8), reused 5 (delta 5), pack-reused 1077 (from 2)
Receiving objects: 100% (1100/1100), 660.70 KiB | 10.83 MiB/s, done.
Resolving deltas: 100% (522/522), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0

In [8]:
# %%writefile /kaggle/working/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_taco_finetune_convnext.yml
# __include__: [
#   '../dataset/coco_detection.yml',
#   '../runtime.yml',
#   './include/dataloader.yml',
#   './include/rtdetrv2_r50vd.yml',
# ]

# output_dir: ./output/rtdetrv2_finetune_taco_convnext_teacher

# RTDETR:
#   backbone: HGNetv2

# HGNetv2:
#   name: 'L'
#   return_idx: [1, 2, 3]
#   freeze_at: 0
#   freeze_norm: True
#   pretrained: True

# task: detection
# remap_mscoco_category: false
# tuning: '../distilled_rtdetr_convnext_teacher_BEST.pth'
# compile: true
# epoches: 50

# num_classes: 60

# train_dataloader:
#   num_workers: 4
#   dataset:
#     type: CocoDetection
#     img_folder: /kaggle/input/dsp-pre-final/processed_taco_coco/train2017
#     ann_file: /kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_train2017.json

# val_dataloader:
#   num_workers: 4
#   dataset:
#     type: CocoDetection
#     img_folder: /kaggle/input/dsp-pre-final/processed_taco_coco/val2017
#     ann_file: /kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_val2017.json

# batch_size: 16

# optimizer:
#   type: AdamW
#   params:
#     - params: '^(?=.*backbone)'
#       lr: 0.00001
#   lr: 0.0001
#   weight_decay: 0.0001
#   betas: [0.9, 0.999]

# lr_scheduler:
#   type: OneCycleLR
#   max_lr: 0.0001
#   pct_start: 0.3
#   total_steps: 4000

# checkpoint_freq: 10

In [9]:
# %cd /kaggle/working/RT-DETR/rtdetrv2_pytorch/

# !torchrun --nproc_per_node=2 tools/train.py \
#     -c configs/rtdetrv2/rtdetrv2_taco_finetune_convnext.yml \
#     --use-amp \
#     --seed=0

In [10]:
# %%writefile convert_vit.py
# import torch
# import glob
# import os
# import sys

# RTDETR_PYTORCH_PATH = '/kaggle/working/RT-DETR/rtdetrv2_pytorch/'
# if RTDETR_PYTORCH_PATH not in sys.path:
#     sys.path.append(RTDETR_PYTORCH_PATH)
# LIGHTLY_CHECKPOINT_DIR = '/kaggle/working/out/distill_vit_lightly/checkpoints/'
# OUTPUT_WEIGHTS_PATH = '/kaggle/working/RT-DETR/distilled_rtdetr_vit_teacher_BEST.pth'

# list_of_files = glob.glob(os.path.join(LIGHTLY_CHECKPOINT_DIR, '*.ckpt'))
# if not list_of_files:
#     raise FileNotFoundError(f"Not found ckpt'{LIGHTLY_CHECKPOINT_DIR}'")
# latest_checkpoint_path = max(list_of_files, key=os.path.getctime)
# lightly_checkpoint = torch.load(latest_checkpoint_path, map_location='cpu', weights_only=False)
# original_state_dict = lightly_checkpoint['state_dict']

# PREFIX_TO_REMOVE = "student_embedding_model.wrapped_model."
# clean_state_dict = {}
# for key, value in original_state_dict.items():
#     if key.startswith(PREFIX_TO_REMOVE):
#         temp_key = key[len(PREFIX_TO_REMOVE):]
#         if temp_key.startswith('_backbone') or temp_key.startswith('_encoder'):
#             new_key = temp_key.lstrip('_')
#             clean_state_dict[new_key] = value

# if not clean_state_dict:
#     raise ValueError("Can not extract")

# final_structure = {'model': clean_state_dict}
# torch.save(final_structure, OUTPUT_WEIGHTS_PATH)

In [11]:
# !python convert_vit.py

In [12]:
# %%writefile /kaggle/working/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_taco_finetune_vit.yml
# __include__: [
#   '../dataset/coco_detection.yml',
#   '../runtime.yml',
#   './include/dataloader.yml',
#   './include/rtdetrv2_r50vd.yml',
# ]

# output_dir: ./output/rtdetrv2_finetune_taco_vit_teacher

# RTDETR:
#   backbone: HGNetv2

# HGNetv2:
#   name: 'L'
#   return_idx: [1, 2, 3]
#   freeze_at: 0
#   freeze_norm: True
#   pretrained: True

# task: detection
# remap_mscoco_category: false
# tuning: '../distilled_rtdetr_vit_teacher_BEST.pth'
# compile: true
# epoches: 50

# num_classes: 60

# train_dataloader:
#   num_workers: 4
#   dataset:
#     type: CocoDetection
#     img_folder: /kaggle/input/dsp-pre-final/processed_taco_coco/train2017
#     ann_file: /kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_train2017.json

# val_dataloader:
#   num_workers: 4
#   dataset:
#     type: CocoDetection
#     img_folder: /kaggle/input/dsp-pre-final/processed_taco_coco/val2017
#     ann_file: /kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_val2017.json

# batch_size: 16

# optimizer:
#   type: AdamW
#   params:
#     - params: '^(?=.*backbone)'
#       lr: 0.00001
#   lr: 0.0001
#   weight_decay: 0.0001
#   betas: [0.9, 0.999]

# lr_scheduler:
#   type: OneCycleLR
#   max_lr: 0.0001
#   pct_start: 0.3
#   total_steps: 4000

# checkpoint_freq: 10

In [13]:
# %cd /kaggle/working/RT-DETR/rtdetrv2_pytorch/

# !torchrun --nproc_per_node=2 tools/train.py \
#     -c configs/rtdetrv2/rtdetrv2_taco_finetune_vit.yml \
#     --use-amp \
#     --seed=0

# With baseline model

In [14]:
# %%writefile /kaggle/working/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_taco_finetune_BASELINE.yml
# __include__: [
#   '../dataset/coco_detection.yml',
#   '../runtime.yml',
#   './include/dataloader.yml',
#   './include/rtdetrv2_r50vd.yml',
# ]

# output_dir: ./output/rtdetrv2_finetune_taco_BASELINE

# RTDETR:
#   backbone: HGNetv2

# HGNetv2:
#   name: 'L'
#   return_idx: [1, 2, 3]
#   freeze_at: 0
#   freeze_norm: True
#   pretrained: True 

# task: detection
# remap_mscoco_category: false

# compile: true
# epoches: 50
# num_classes: 60


# train_dataloader:
#   num_workers: 4
#   dataset:
#     type: CocoDetection
#     img_folder: /kaggle/input/dsp-pre-final/processed_taco_coco/train2017
#     ann_file: /kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_train2017.json

# val_dataloader:
#   num_workers: 4
#   dataset:
#     type: CocoDetection
#     img_folder: /kaggle/input/dsp-pre-final/processed_taco_coco/val2017
#     ann_file: /kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_val2017.json

# batch_size: 16

# optimizer:
#   type: AdamW
#   params:
#     - params: '^(?=.*backbone)'
#       lr: 0.00001
#   lr: 0.0001
#   weight_decay: 0.0001
#   betas: [0.9, 0.999]

# lr_scheduler:
#   type: OneCycleLR
#   max_lr: 0.0001
#   pct_start: 0.3
#   total_steps: 4000

# checkpoint_freq: 10

In [15]:
# %cd /kaggle/working/RT-DETR/rtdetrv2_pytorch/

# !torchrun --nproc_per_node=2 tools/train.py \
#     -c configs/rtdetrv2/rtdetrv2_taco_finetune_BASELINE.yml \
#     --use-amp \
#     --seed=0

In [16]:
!pip install -q ultralytics
%cd /kaggle/working

/kaggle/working


In [17]:
import os
import json
from tqdm import tqdm
import yaml

YOLO_DATA_ROOT = '/kaggle/working/taco_yolo'
IMAGE_TRAIN_DIR_DEST = os.path.join(YOLO_DATA_ROOT, 'images', 'train')
IMAGE_VAL_DIR_DEST = os.path.join(YOLO_DATA_ROOT, 'images', 'val')
LABEL_TRAIN_DIR_DEST = os.path.join(YOLO_DATA_ROOT, 'labels', 'train')
LABEL_VAL_DIR_DEST = os.path.join(YOLO_DATA_ROOT, 'labels', 'val')

os.makedirs(IMAGE_TRAIN_DIR_DEST, exist_ok=True)
os.makedirs(IMAGE_VAL_DIR_DEST, exist_ok=True)
os.makedirs(LABEL_TRAIN_DIR_DEST, exist_ok=True)
os.makedirs(LABEL_VAL_DIR_DEST, exist_ok=True)

IMAGE_TRAIN_DIR_SRC = '/kaggle/input/dsp-pre-final/processed_taco_coco/train2017'
IMAGE_VAL_DIR_SRC = '/kaggle/input/dsp-pre-final/processed_taco_coco/val2017'

!cp -n {IMAGE_TRAIN_DIR_SRC}/* {IMAGE_TRAIN_DIR_DEST}/
!cp -n {IMAGE_VAL_DIR_SRC}/* {IMAGE_VAL_DIR_DEST}/

COCO_ANNOTATIONS_TRAIN = '/kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_train2017.json'
COCO_ANNOTATIONS_VAL = '/kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_val2017.json'

def convert_coco_to_yolo(json_file, output_labels_dir):
    with open(json_file) as f: data = json.load(f)
    images_map = {img['id']: (img['file_name'], img['width'], img['height']) for img in data['images']}
    for ann in tqdm(data['annotations'], desc=f"Converting {os.path.basename(json_file)}"):
        image_id, class_id = ann['image_id'], ann['category_id']
        if image_id not in images_map: continue
        file_name, img_w, img_h = images_map[image_id]
        box = ann['bbox']
        x, y, w, h = box
        x_center, y_center = (x + w / 2) / img_w, (y + h / 2) / img_h
        norm_w, norm_h = w / img_w, h / img_h
        label_file_name = os.path.splitext(file_name)[0] + '.txt'
        label_file_path = os.path.join(output_labels_dir, label_file_name)
        with open(label_file_path, 'a') as f:
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}\n")

convert_coco_to_yolo(COCO_ANNOTATIONS_TRAIN, LABEL_TRAIN_DIR_DEST)
convert_coco_to_yolo(COCO_ANNOTATIONS_VAL, LABEL_VAL_DIR_DEST)

with open(COCO_ANNOTATIONS_TRAIN) as f: coco_data = json.load(f)
categories = sorted(coco_data['categories'], key=lambda x: x['id'])
class_names = [cat['name'] for cat in categories]

taco_yaml_content = {
    'path': YOLO_DATA_ROOT, 'train': 'images/train', 'val': 'images/val',
    'nc': len(class_names), 'names': class_names
}

YAML_PATH = os.path.join(YOLO_DATA_ROOT, 'taco.yaml')
with open(YAML_PATH, 'w') as f: yaml.dump(taco_yaml_content, f, sort_keys=False)

Converting instances_val2017.json: 100%|██████████| 776/776 [00:00<00:00, 32674.58it/s]


In [18]:
import yaml
import json
import os

COCO_ANNOTATIONS_TRAIN = '/kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_train2017.json'
YOLO_DATA_ROOT = '/kaggle/working/taco_yolo'

with open(COCO_ANNOTATIONS_TRAIN) as f:
    coco_data = json.load(f)

categories = sorted(coco_data['categories'], key=lambda x: x['id'])
class_names = [cat['name'] for cat in categories]

taco_yaml_content = {
    'path': YOLO_DATA_ROOT,
    'train': 'images/train',
    'val': 'images/val',
    'nc': len(class_names),
    'names': class_names
}

YAML_PATH = os.path.join(YOLO_DATA_ROOT, 'taco.yaml')
with open(YAML_PATH, 'w') as f:
    yaml.dump(taco_yaml_content, f, sort_keys=False)

with open(YAML_PATH, 'r') as f:
    print(f.read())

path: /kaggle/working/taco_yolo
train: images/train
val: images/val
nc: 60
names:
- Aluminium foil
- Battery
- Aluminium blister pack
- Carded blister pack
- Other plastic bottle
- Clear plastic bottle
- Glass bottle
- Plastic bottle cap
- Metal bottle cap
- Broken glass
- Food Can
- Aerosol
- Drink can
- Toilet tube
- Other carton
- Egg carton
- Drink carton
- Corrugated carton
- Meal carton
- Pizza box
- Paper cup
- Disposable plastic cup
- Foam cup
- Glass cup
- Other plastic cup
- Food waste
- Glass jar
- Plastic lid
- Metal lid
- Other plastic
- Magazine paper
- Tissues
- Wrapping paper
- Normal paper
- Paper bag
- Plastified paper bag
- Plastic film
- Six pack rings
- Garbage bag
- Other plastic wrapper
- Single-use carrier bag
- Polypropylene bag
- Crisp packet
- Spread tub
- Tupperware
- Disposable food container
- Foam food container
- Other plastic container
- Plastic glooves
- Plastic utensils
- Pop tab
- Rope & strings
- Scrap metal
- Shoe
- Squeezable tube
- Plastic straw


In [19]:
%%writefile training_yolov11.py
from ultralytics import YOLO
import wandb
import os

output_path = '/kaggle/working/yolo_checkpoints'
os.makedirs(output_path, exist_ok=True)

try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    wandb_key = secrets.get_secret("WANDB_API_KEY")
    wandb.login(key=wandb_key)
except Exception as e:
    print(f"{e}")
    
wandb.init(
    project='yolo_runs_taco',
    name='yolo11l_taco_finetune_baseline',
    job_type='fine-tuning'
)

model = YOLO('yolo11l.pt')

results = model.train(
    data='/kaggle/working/taco_yolo/taco.yaml',
    epochs=50,
    imgsz=640,
    batch=32,
    project=output_path,
    name='yolo11l_finetune_baseline',
    exist_ok=True,
    device=[0, 1]
)
wandb.finish()

Writing training_yolov11.py


In [20]:
!python training_yolov11.py

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' a